In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from functools import reduce
 
import torch
import torchvision
from torchinfo import summary
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

torch.manual_seed(87)

device = torch.device('cpu')

In [2]:
# The mean and std are kept as 0.5 for normalizing pixel values as the pixel values are originally in the range 0 to 1
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomCrop(32, 4),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5),
                                                           (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(
    root=os.path.join('/Users/pepijnschouten/Desktop/Python_Scripts/' \
        'Python_Scripts_Books/Deep_Learning/Mastering_Pytorch/Datasets/' \
            'CIFAR10', 'data'),
    train=True, download=True, transform=train_transform)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=16, shuffle=True, num_workers=1)


test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR10(
    root=os.path.join('/Users/pepijnschouten/Desktop/Python_Scripts/' \
        'Python_Scripts_Books/Deep_Learning/Mastering_Pytorch/Datasets/' \
            'CIFAR10', 'data'),
    train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=10000, shuffle=False, num_workers=2)


# ordering is important
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class LeNet(nn.Module):
    def __init__(self, num_classes):
        super(LeNet, self).__init__()
        
        self.cn1 = nn.Conv2d(3, 6, 5)
        self.cn2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        x = F.relu(self.cn1(x))
        x = F.max_pool2d(x, 2, 2)
        
        x = F.relu(self.cn2(x))
        x = F.max_pool2d(x, 2, 2)

        x = x.view(-1, self.featured_size(x))
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def featured_size(self, x):
        size = x.size()[1:]  # all except the first (batch) dimension
        return reduce(lambda x, y: x * y, size)

lenet = LeNet(num_classes=10).to(device)
print(lenet)
print(summary(lenet, (1, 3, 32, 32)))


LeNet(
  (cn1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (cn2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
Layer (type:depth-idx)                   Output Shape              Param #
LeNet                                    [1, 10]                   --
├─Conv2d: 1-1                            [1, 6, 28, 28]            456
├─Conv2d: 1-2                            [1, 16, 10, 10]           2,416
├─Linear: 1-3                            [1, 120]                  48,120
├─Linear: 1-4                            [1, 84]                   10,164
├─Linear: 1-5                            [1, 10]                   850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.66
Input size (MB): 0.01
Forward/backward pass size (MB): 0.05
Params size (MB

In [4]:
def train(net, trainloader, optim, epoch):
    loss_total = 0.
    
    for i, data in enumerate(trainloader, 0):
        in_img, ground_truth = data
        in_img = in_img.to(device)
        ground_truth = ground_truth.to(device)
        
        optim.zero_grad()
        
        op = net(in_img)
        loss = nn.CrossEntropyLoss()(op, ground_truth)
        loss.backward()
        optim.step()
        
        loss_total += loss.item()
        
        if (i + 1) % 1000 == 0:
            print('[Epoch: {:3}, Mini-batch: {:5}] loss: {:.4f}'.format(epoch + 1,
                                             i + 1,
                                             loss_total / 200))
            loss_total = 0.

In [5]:
def test(net, testloader):
    success = 0
    counter = 0
    
    with torch.inference_mode():
        for data in testloader:
            in_img, ground_truth = data
            in_img = in_img.to(device)
            ground_truth = ground_truth.to(device)
            
            op = net(in_img)
            _, pred = torch.max(op.data, 1)
            counter += ground_truth.size(0)
            success += (pred == ground_truth).sum().item()

    print('LeNet accuracy on 10000 images from test dataset: {:.2f}%'.format(
        100 * success / counter))

In [6]:
# define optimizer
optim = torch.optim.Adam(lenet.parameters(), lr=0.001)

# training loop over the dataset multiple times
epochs = 15
print('Started Training')
for epoch in range(epochs):  
    train(lenet, trainloader, optim, epoch)
    print()
    test(lenet, testloader)
    print()

print('Finished Training / Saving model')
model_path = os.path.join(
    '/Users/pepijnschouten/Desktop/Python_Scripts/' \
        'Python_Scripts_Books/Deep_Learning/' \
            'Mastering_Pytorch/Own_Files/Ch3_Deep_CNNs/' \
                'model_dir',
    'lenet_cifar_model.pth')
torch.save(lenet.state_dict(), model_path)

Started Training
[Epoch:   1, Mini-batch:  1000] loss: 9.2186
[Epoch:   1, Mini-batch:  2000] loss: 8.1888
[Epoch:   1, Mini-batch:  3000] loss: 7.7685

LeNet accuracy on 10000 images from test dataset: 45.46%

[Epoch:   2, Mini-batch:  1000] loss: 7.4541
[Epoch:   2, Mini-batch:  2000] loss: 7.1889
[Epoch:   2, Mini-batch:  3000] loss: 6.9728

LeNet accuracy on 10000 images from test dataset: 53.90%

[Epoch:   3, Mini-batch:  1000] loss: 6.8295
[Epoch:   3, Mini-batch:  2000] loss: 6.7022
[Epoch:   3, Mini-batch:  3000] loss: 6.6263

LeNet accuracy on 10000 images from test dataset: 56.25%

[Epoch:   4, Mini-batch:  1000] loss: 6.4609
[Epoch:   4, Mini-batch:  2000] loss: 6.5063
[Epoch:   4, Mini-batch:  3000] loss: 6.3016

LeNet accuracy on 10000 images from test dataset: 58.45%

[Epoch:   5, Mini-batch:  1000] loss: 6.2600
[Epoch:   5, Mini-batch:  2000] loss: 6.1711
[Epoch:   5, Mini-batch:  3000] loss: 6.1169

LeNet accuracy on 10000 images from test dataset: 59.79%

[Epoch:   6, 

In [7]:
# load test dataset images
d_iter = iter(testloader)
im, ground_truth = next(d_iter)

# load model
lenet_cached = LeNet(num_classes=10)
lenet_cached.load_state_dict(torch.load(model_path,
                                        weights_only=True))

<All keys matched successfully>

In [8]:
success = 0
counter = 0
with torch.no_grad():
    for data in testloader:
        im, ground_truth = data
        op = lenet_cached(im)
        _, pred = torch.max(op.data, 1)
        counter += ground_truth.size(0)
        success += (pred == ground_truth).sum().item()

print('Model accuracy on 10000 images from test dataset: %d %%' % (
    100 * success / counter))

Model accuracy on 10000 images from test dataset: 64 %


In [9]:
class_success = list(0. for i in range(10))
class_counter = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        im, ground_truth = data
        op = lenet_cached(im)
        _, pred = torch.max(op, 1)
        c = (pred == ground_truth).squeeze()
        for i in range(10000):
            ground_truth_curr = ground_truth[i]
            class_success[ground_truth_curr] += c[i].item()
            class_counter[ground_truth_curr] += 1


for i in range(10):
    print('Model accuracy for class %5s : %2d %%' % (
        classes[i], 100 * class_success[i] / class_counter[i]))

Model accuracy for class plane : 73 %
Model accuracy for class   car : 91 %
Model accuracy for class  bird : 55 %
Model accuracy for class   cat : 39 %
Model accuracy for class  deer : 57 %
Model accuracy for class   dog : 52 %
Model accuracy for class  frog : 81 %
Model accuracy for class horse : 66 %
Model accuracy for class  ship : 69 %
Model accuracy for class truck : 56 %
